# 37009 Workshop Week 4: Risk Factor Mapping

## Some Useful Functions

The following functions shall be useful in carrying out some of the calculations in the exercises. Specifically, we recall the option prices and Greeks under the Black-Scholes-Merton model. In addition, we construct a two-vertex cash flow mapping function.

In [2]:
# Required libraries
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import math

### Black-Scholes-Merton Formulas for European Put and Call Options

In [3]:
# Pricing function for European calls and puts
def BSprice(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option prices
    if option_type == 'call':
        price = (np.exp(-q * tau) * s * sp.stats.norm.cdf(d1, 0.0, 1.0) \
                 - np.exp(-r * tau) * K * sp.stats.norm.cdf(d2, 0.0, 1.0))
        
    if option_type == 'put':
        price = (np.exp(-r * tau) * K * sp.stats.norm.cdf(-d2, 0.0, 1.0) \
                 - np.exp(-q * tau) * s * sp.stats.norm.cdf(-d1, 0.0, 1.0))
        
    return price

In [4]:
# Function to calculate option delta under the Black-Scholes-Merton model
def BSdelta(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    
    # Calculate option delta
    if option_type == 'call':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0)
    
    if option_type == 'put':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0) - 1
    
    return value    

In [5]:
# Function to calculate option gamma under the Black-Scholes-Merton model
def BSgamma(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Compute option gamma (same for calls and puts)
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    value = sp.stats.norm.pdf(d1, 0.0, 1.0) / (s * sigma * np.sqrt(tau))
    
    return value

### Cash Flow Mapping (PV and Duration or Volatility Invariant Maps with Two Vertices)

In [7]:
# Cash flow mapping using two vertices
def CFmap2V(maturity, left_maturity, right_maturity, map_type = "Duration", 
            left_vol = None, right_vol = None, left_right_cor = None):
    
    # Assumption: Cash flow at non-standard maturity is mapped to two vertices, one on the left and one on the right.
    # Imposes PV-invariance; may impose duration-invariance or volatility-invariance, but not both.
    
    # Output: alpha parameter to allocate a PV of 1 to the left vertex T1. 
    # For the volatility mapping, the function returns two values of alpha. 
    # We often will choose the alpha that is between 0 and 1.
    
    # Use mathematical notation for function inputs
    T = maturity
    T1 = left_maturity
    T2 = right_maturity
    
    if map_type == "Duration":
        alpha = (T2 - T) / (T2 - T1)
        return alpha
    
    if map_type == "Volatility":
        
        if (left_vol == None or right_vol == None or left_right_cor == None):
            print("Please provide non-empty volatility-invariant mapping inputs")
        
        else:
            # Extract volatility-invariant CF map inputs
            sigma1 = left_vol
            sigma2 = right_vol
            rho = left_right_cor
            
            # Linearly interpolate the variance at the non-standard maturity
            sq_sigma_interp = np.interp(x = T, xp = np.array([T1, T2]), fp = np.array([sigma1 ** 2, sigma2 ** 2]))
            
            # Coefficients of quadratic equation for alpha
            a_coeff = sigma1 ** 2 - 2 * rho * sigma1 * sigma2 + sigma2 ** 2
            b_coeff = 2 * rho * sigma1 * sigma2 - 2 * sigma2 ** 2
            c_coeff = sigma2 ** 2 - sq_sigma_interp
            
            # Solve quadratic equation for alpha
            alpha1 = (-b_coeff + np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha2 = (-b_coeff - np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha = np.array([alpha1, alpha2])
            
            return alpha